### CAFR10

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision
import torch.optim as optim
from datagen import *
from nets import *
from backdoor import Backdoor
from defense import Defense

# move to GPU (if applicable)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/andmholm/Backdoor/backdoor_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Choosing Network and Dataset

In [2]:
# choose network configuration and dataset
backdoor = Backdoor(VggNet, config='16-layer', channels=3, classes=10, dropout=0.5)
backdoor.create_models(optim.SGD, nn.CrossEntropyLoss, lr=0.01)
backdoor.load_data(torchvision.datasets.CIFAR10, ImageEntity)

Files already downloaded and verified
Files already downloaded and verified


### Poison setup

In [3]:
# create patches, target labels, and define transformations/merges 
patches = ImagePatch((9, 9), 3, 'random').get_patches(len(backdoor))
targets = {i : i + 1 if i < 9 else 0 for i in range(10)}
transforms = (None, [RotateTransform()], [LabelTransform(targets)])
merge = ImageMerge(select=True)

### Poisoning

In [4]:
# poison 20% of the data
backdoor.poison(patches, transforms, merge, pct=0.2)

### Training Base and Trojan Net

In [5]:
# train the base DNN & the trojan DNN 
loss = backdoor.train(2, epochs=3, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/3 | 24.99% | Loss: 1.8116 | Samples trained: 12496/50000
Epoch 1/3 | 49.98% | Loss: 1.6237 | Samples trained: 24992/50000
Epoch 1/3 | 74.98% | Loss: 1.4911 | Samples trained: 37488/50000
Epoch 1/3 | 99.97% | Loss: 1.3895 | Samples trained: 49984/50000
Epoch 1 complete | Loss: 1.3897
Epoch 2/3 | 24.99% | Loss: 0.9699 | Samples trained: 12496/50000
Epoch 2/3 | 49.98% | Loss: 0.9351 | Samples trained: 24992/50000
Epoch 2/3 | 74.98% | Loss: 0.8919 | Samples trained: 37488/50000
Epoch 2/3 | 99.97% | Loss: 0.8551 | Samples trained: 49984/50000
Epoch 2 complete | Loss: 0.8550
Epoch 3/3 | 24.99% | Loss: 0.6927 | Samples trained: 12496/50000
Epoch 3/3 | 49.98% | Loss: 0.6704 | Samples trained: 24992/50000
Epoch 3/3 | 74.98% | Loss: 0.6453 | Samples trained: 37488/50000
Epoch 3/3 | 99.97% | Loss: 0.6219 | Samples trained: 49984/50000
Epoch 3 complete | Loss: 0.6219
Training complete | Net Average Loss: 0.9555 | Total epochs: 3
Training started
Epoch 1/3 | 24.99% | Loss: 

### Evaluating Backdoor

In [6]:
# evulate backdoor based on VA & ASR
metrics = backdoor.eval(verbose=True, device=device)

Accuracy on Clean | Base 77.45% | Trojan 74.80% | Difference -2.65%
Base Accuracy on Poison 4.21% | Attack Success Rate (ASR): 66.72%
Average Tensor Distance: 309.48 | Net Tensor Difference 15473881.31


### Creating a Defense

In [7]:
# init the defense
defense = Defense(backdoor)

### Detection setup

In [8]:
# get base & trojan DNN, both clean & poisoned entitysets, sizes ranges to create synthetic patch
base, trojan = backdoor.get_net_modules()
cleantrain, poisontrain, cleantest, poisontest = backdoor.get_datasets()
patch_size_ranges = [(3, 3), (5, 5), (7, 7), (9, 9), (11, 11)]

### Detection Base

In [9]:
# detect backdoor with 30% of the clean samples poisoned and detect based on a 10% drop in VA for base DNN
metrics = defense.detect(base, cleantest, threshold=0.1, size_ranges=patch_size_ranges,  pct=0.3, verbose=True, device=device)

Patch size (3, 3) | Synthetic Poison Accuracy 77.17% | Original Accuracy 77.45% | Difference -0.28%
Patch size (5, 5) | Synthetic Poison Accuracy 76.47% | Original Accuracy 77.45% | Difference -0.98%
Patch size (7, 7) | Synthetic Poison Accuracy 75.70% | Original Accuracy 77.45% | Difference -1.75%
Patch size (9, 9) | Synthetic Poison Accuracy 74.96% | Original Accuracy 77.45% | Difference -2.49%
Patch size (11, 11) | Synthetic Poison Accuracy 72.66% | Original Accuracy 77.45% | Difference -4.79%
Average Accuracy 75.39% | Average Difference -2.06% | Lowest Score: 72.66% | Likihood of Backdoor: Low


### Detection Trojan

In [10]:
# detect backdoor with 30% of the clean samples poisoned and detect based on a 10% drop in VA for trojan DNN
metrics = defense.detect(trojan, cleantest, threshold=0.1, size_ranges=patch_size_ranges,  pct=0.3, verbose=True, device=device)

Patch size (3, 3) | Synthetic Poison Accuracy 74.66% | Original Accuracy 74.80% | Difference -0.14%
Patch size (5, 5) | Synthetic Poison Accuracy 73.63% | Original Accuracy 74.80% | Difference -1.17%
Patch size (7, 7) | Synthetic Poison Accuracy 60.37% | Original Accuracy 74.80% | Difference -14.43%
Patch size (9, 9) | Synthetic Poison Accuracy 56.04% | Original Accuracy 74.80% | Difference -18.76%
Patch size (11, 11) | Synthetic Poison Accuracy 53.40% | Original Accuracy 74.80% | Difference -21.40%
Average Accuracy 63.62% | Average Difference -11.18% | Lowest Score: 53.40% | Likihood of Backdoor: High


### Blocking Patches setup

In [11]:
# reverse labels (for research VA purposes), create patch to compare variances with in parts of images (same size as patch), define samples to block
patch = ImagePatch((10, 10), 3, 'random')
n = 5000

### Blocking

In [12]:
# block poisoned images with average rgb values the same dimensions as the patch
blockedloader = defense.block(poisontest, patch, n=n)

### Testing after Block Base

In [13]:
# retest after blocking for base DNN
metrics = defense.test(base, blockedloader, verbose=True, device=device)

Testing started
24.46% Testing complete | Loss: 6.0120 | Accuracy: 0.0312
48.92% Testing complete | Loss: 5.9516 | Accuracy: 0.0363
73.38% Testing complete | Loss: 5.9648 | Accuracy: 0.0325
97.84% Testing complete | Loss: 5.9878 | Accuracy: 0.0352
Testing complete | Loss: 5.9882 | Accuracy: 3.56%


### Testing after Block Trojan

In [14]:
# retest after blocking for trojan DNN
metrics = defense.test(trojan, blockedloader, verbose=True, device=device)

Testing started
24.46% Testing complete | Loss: 4.5737 | Accuracy: 0.1608
48.92% Testing complete | Loss: 4.5439 | Accuracy: 0.1613
73.38% Testing complete | Loss: 4.4793 | Accuracy: 0.1688
97.84% Testing complete | Loss: 4.4875 | Accuracy: 0.1712
Testing complete | Loss: 4.4897 | Accuracy: 17.10%


### Defense by retraining

In [15]:
# get a clean dataloader, reset the defense net module, retrain the defense net model (og trojan model)
dataloader = cleantrain.get_dataloader()
defense.reset()
loss = defense.retrain(dataloader, epochs=3, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/3 | 24.95% | Loss: 0.8478 | Samples trained: 12480/50000
Epoch 1/3 | 49.90% | Loss: 0.8428 | Samples trained: 24960/50000
Epoch 1/3 | 74.86% | Loss: 0.8284 | Samples trained: 37440/50000
Epoch 1/3 | 99.81% | Loss: 0.7874 | Samples trained: 49920/50000
Epoch 1 complete | Loss: 0.7870
Epoch 2/3 | 24.95% | Loss: 0.8464 | Samples trained: 12480/50000
Epoch 2/3 | 49.90% | Loss: 0.8386 | Samples trained: 24960/50000
Epoch 2/3 | 74.86% | Loss: 0.8231 | Samples trained: 37440/50000
Epoch 2/3 | 99.81% | Loss: 0.7831 | Samples trained: 49920/50000
Epoch 2 complete | Loss: 0.7828
Epoch 3/3 | 24.95% | Loss: 0.8508 | Samples trained: 12480/50000
Epoch 3/3 | 49.90% | Loss: 0.8415 | Samples trained: 24960/50000
Epoch 3/3 | 74.86% | Loss: 0.8246 | Samples trained: 37440/50000
Epoch 3/3 | 99.81% | Loss: 0.7851 | Samples trained: 49920/50000
Epoch 3 complete | Loss: 0.7848
Training complete | Net Average Loss: 0.7849 | Total epochs: 3


### Evaluation after retraining

In [16]:
# re-evaluate the trojan DNN
cleanloader, poisonloader = cleantest.get_dataloader(), poisontest.get_dataloader()
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 77.45% | Trojan 74.80% | Defense 69.93%
Accuracy on Posion | Base 4.21% | Defense 40.37% | Trojan ASR 66.72%
Difference from Baseline | Trojan -2.65% | Defense -7.52%
Defense Effectiveness | 26.35% decrease in ASR


In [17]:
defense.view_named_modules()

avgpool: AdaptiveAvgPool2d(output_size=(7, 7))
drop: Dropout(p=0.5, inplace=False)
fc1: Linear(in_features=25088, out_features=4096, bias=True)
fc2: Linear(in_features=4096, out_features=4096, bias=True)
fc3: Linear(in_features=4096, out_features=10, bias=True)
layers: Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): MaxPool2d(kernel_size=(2, 2), stride=2, padding=1, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running

### Pruning setup

In [18]:
# determine layers to prune, decide percent of neurons to prune, reset the defense (original trojan DNN)
layers = ['layers.28', 'layers.26', 'layers.24', 'layers.21']
amount = 0.5
defense.reset()

### Pruning based on l-Infinity norm of weights from layers

In [19]:
# prune 50% the weights of last 4 convolutional layers based on the l-Infinity norm
defense.prune(layers, amount)

### Evaluation after prune

In [20]:
# evaluate the model after pruning
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 77.45% | Trojan 74.80% | Defense 12.92%
Accuracy on Posion | Base 4.21% | Defense 8.72% | Trojan ASR 66.72%
Difference from Baseline | Trojan -2.65% | Defense -64.53%
Defense Effectiveness | 58.00% decrease in ASR


### Retraining to recover

In [21]:
# retrain to recover
loss = defense.retrain(dataloader, epochs=3, verbose=True, device=device, batch_size=16)

Training started
Epoch 1/3 | 24.95% | Loss: 1.4669 | Samples trained: 12480/50000
Epoch 1/3 | 49.90% | Loss: 1.4700 | Samples trained: 24960/50000
Epoch 1/3 | 74.86% | Loss: 1.4675 | Samples trained: 37440/50000
Epoch 1/3 | 99.81% | Loss: 1.4559 | Samples trained: 49920/50000
Epoch 1 complete | Loss: 1.4558
Epoch 2/3 | 24.95% | Loss: 1.4724 | Samples trained: 12480/50000
Epoch 2/3 | 49.90% | Loss: 1.4717 | Samples trained: 24960/50000
Epoch 2/3 | 74.86% | Loss: 1.4685 | Samples trained: 37440/50000
Epoch 2/3 | 99.81% | Loss: 1.4575 | Samples trained: 49920/50000
Epoch 2 complete | Loss: 1.4575
Epoch 3/3 | 24.95% | Loss: 1.4713 | Samples trained: 12480/50000
Epoch 3/3 | 49.90% | Loss: 1.4710 | Samples trained: 24960/50000
Epoch 3/3 | 74.86% | Loss: 1.4684 | Samples trained: 37440/50000
Epoch 3/3 | 99.81% | Loss: 1.4568 | Samples trained: 49920/50000
Epoch 3 complete | Loss: 1.4568
Training complete | Net Average Loss: 1.4567 | Total epochs: 3


### Evaluation after retraining

In [22]:
metrics = defense.eval(cleanloader, poisonloader, verbose=True, device=device)

Accuracy on clean | Base 77.45% | Trojan 74.80% | Defense 61.85%
Accuracy on Posion | Base 4.21% | Defense 41.35% | Trojan ASR 66.72%
Difference from Baseline | Trojan -2.65% | Defense -15.60%
Defense Effectiveness | 25.37% decrease in ASR
